In [26]:
# 南璃 字段内容explode,转成多行
def concatBarcodes(path):
    import pandas as pd
    import openpyxl
    # /Users/huangjiyong/Desktop/大佬帮帮忙~.xlsx
    tab = pd.read_excel(path)
    tab['颜色'] = tab['颜色'].map(lambda x: x.split(','))
    tab['规格'] = tab['规格'].map(lambda x: x.split(','))
    tab = tab.explode('颜色') .explode('规格')
    tab['条码'] = tab['商品'] + tab['颜色'] + tab['规格']
    tab = tab.reset_index(drop=True)
    writer = pd.ExcelWriter(path,engine = 'openpyxl') 
    book = openpyxl.load_workbook(writer.path)
    writer.book = book
    tab.to_excel(writer,index=False,sheet_name='条码')
    writer.save()
    writer.close()
    return tab.shape
concatBarcodes('/Users/huangjiyong/Desktop/大佬帮帮忙~.xlsx')

(45, 44)

In [2]:
import pandas as pd

In [3]:
tab = pd.read_excel('/Users/huangjiyong/coding/project/vscodeproject/jupyter/tmpCoding/模版-GMV周实际.xlsx')

In [4]:
tab.columns

Index(['事业部', '业务类型', '经营主体', '品牌', '类目', '渠道', '入参日期', '年度', '周数', 'GMV'], dtype='object')